
# 📊 Automação e Análise de Dados de Contratos e Convênios

Este notebook demonstra como automatizar a **coleta, o tratamento e a análise** de dados de contratos e convênios disponibilizados pela plataforma **Ceará Transparente**.

**Objetivos:**
- Coletar dados via API.
- Estruturar e armazenar os dados.
- Analisar e visualizar as informações.
- Exportar resultados tratados.
- Enviar automaticamente por e-mail todos os dias.


## 1️⃣ Importação de Bibliotecas

In [4]:

import requests        # Para consumir a API
import pandas as pd    # Para manipulação e análise de dados
import matplotlib.pyplot as plt  # Para criar gráficos


## 2️⃣ Coleta de Dados via API

In [ ]:

# URL da API do Ceará Transparente (exemplo hipotético)
URL_API = "https://api-dados-abertos.cearatransparente.ce.gov.br/transparencia/contratos/contratos?page=20&data_assinatura_inicio=01%2F01%2F2024&data_assinatura_fim=01%2F01%2F2025"

# Fazendo a requisição dos dados
response = requests.get(URL_API)

# Verificando se a resposta foi bem-sucedida
if response.status_code == 200:
    dados = response.json()  # Converte o resultado para um dicionário
    print("✅ Conexão com a API realizada com sucesso!")
else:
    print(f"❌ Erro ao acessar API: {response.status_code}")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## 3️⃣ Transformação dos Dados em DataFrame

In [ ]:

# Converte os dados para um DataFrame do Pandas
df = pd.DataFrame(dados)

# Exibe as primeiras 5 linhas para conferir o carregamento
df.head()


## 4️⃣ Análise Exploratória de Dados (AED)

In [ ]:

# Mostra estatísticas descritivas dos valores de contrato
df["valor_contrato"].describe()


## 5️⃣ Visualização dos Dados

In [ ]:

# Gera um histograma da distribuição de valores de contrato
plt.figure(figsize=(10, 5))
plt.hist(df["valor_contrato"], bins=30)
plt.title("Distribuição dos Valores de Contrato")
plt.xlabel("Valor (R$)")
plt.ylabel("Frequência")
plt.grid()
plt.show()


## 6️⃣ Exportação dos Dados Tratados

In [ ]:

# Salva os dados tratados em CSV
df.to_csv("contratos_tratados.csv", index=False)
print("📂 Arquivo 'contratos_tratados.csv' salvo com sucesso!")


## 7️⃣ Envio Automático de Relatório por E-mail

In [ ]:

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

# --- CONFIGURAÇÕES DO E-MAIL ---
EMAIL_REMETENTE = "seuemail@gmail.com"
SENHA_REMETENTE = "sua_senha_de_aplicativo"  # Use senha de app, não a senha normal!
EMAIL_DESTINATARIO = "destinatario@exemplo.com"

# --- CRIAÇÃO DA MENSAGEM ---
mensagem = MIMEMultipart()
mensagem['From'] = EMAIL_REMETENTE
mensagem['To'] = EMAIL_DESTINATARIO
mensagem['Subject'] = "📊 Relatório Diário de Contratos e Convênios"

corpo = """
<p>Olá,</p>
<p>Segue em anexo o relatório diário de contratos e convênios extraídos da plataforma Ceará Transparente.</p>
<p>Atenciosamente,<br>Automação Python</p>
"""

mensagem.attach(MIMEText(corpo, 'html'))

# --- ANEXANDO O CSV GERADO ---
arquivo = "contratos_tratados.csv"
with open(arquivo, "rb") as anexo:
    parte = MIMEBase('application', 'octet-stream')
    parte.set_payload(anexo.read())
    encoders.encode_base64(parte)
    parte.add_header('Content-Disposition', f'anexo; filename={os.path.basename(arquivo)}')
    mensagem.attach(parte)

# --- ENVIO DO E-MAIL ---
try:
    with smtplib.SMTP('smtp.gmail.com', 587) as servidor:
        servidor.starttls()
        servidor.login(EMAIL_REMETENTE, SENHA_REMETENTE)
        servidor.send_message(mensagem)
        print("✅ E-mail enviado com sucesso!")
except Exception as e:
    print(f"❌ Erro ao enviar e-mail: {e}")



## ✅ Conclusão

- A coleta de dados foi realizada com sucesso via API.
- Os dados foram estruturados em um DataFrame, analisados e visualizados.
- Um arquivo CSV com os dados tratados foi gerado para uso posterior.
- O relatório foi enviado automaticamente por e-mail para os responsáveis.
- Para execução diária, configure o agendamento no Windows (Task Scheduler) ou Linux (cron).
